In [1]:
import math
import numpy as np
import pandas as pd

import json

import os

import gmaps
import gmaps.geojson_geometries

from geographiclib.geodesic import Geodesic

import re
import pprint


import psycopg2

In [2]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [3]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    return(pd.DataFrame(cursor.fetchall(), columns =['zip', 'population']))

In [4]:
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [5]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [6]:
cursor = connection.cursor()

In [7]:
downtown_berkley_1_mile = my_station_get_zips('Downtown Berkeley', 1)
downtown_berkley_3_mile = my_station_get_zips('Downtown Berkeley', 3)
downtown_berkley_5_mile = my_station_get_zips('Downtown Berkeley', 5)
downtown_berkley_10_mile = my_station_get_zips('Downtown Berkeley', 10)

downtown_berkley_1_mile

,zip,population
0,94702,17092
1,94703,21937
2,94704,29190
3,94709,11740
4,94720,2971


In [8]:
def get_stations():
    connection.rollback()
    
    query = "select station from stations where station <> 'Downtown Berkley'"

    cursor.execute(query)

    stations = []
    temp_stations = cursor.fetchall()
    for station in temp_stations:
        stations.append(station[0])
    
    return stations

In [9]:
def find_net_new_customers(distance):
    """finds all stations where 50% of the population within the distence would be net new"""
    stations = get_stations()
    
    berkeley_zips = my_station_get_zips('Downtown Berkeley', distance)
    berkeley_pop = berkeley_zips['population'].sum()
    
    new_stations = []
    
    for station in stations:
        station_zips = my_station_get_zips(station, distance)
        
        new_pop = station_zips['population'].sum()
        overlap_zips = pd.DataFrame(berkeley_zips.merge(station_zips, right_on='zip', left_on='zip',how='inner'))
        
        overlap_zip_pop = overlap_zips['population_x'].sum()
        
        percent_new = ( new_pop / overlap_zip_pop ) if overlap_zip_pop != 0 else 0
        
        if percent_new < .75:
            new_stations.append(station)
    
    return new_stations

In [10]:
find_net_new_customers(5)

['16th Street Mission',
 '24th Street Mission',
 'Antioch',
 'Balboa Park',
 'Bay Fair',
 'Berryessa',
 'Castro Valley',
 'Civic Center',
 'Colma',
 'Concord',
 'Daly City',
 'Dublin',
 'Embarcadero',
 'Fremont',
 'Glen Park',
 'Hayward',
 'Millbrae',
 'Milpitas',
 'Montgomery Street',
 'North Concord',
 'OAK',
 'Pittsburg',
 'Pittsburg Center',
 'Pleasant Hill',
 'Powell Street',
 'SFO',
 'San Bruno',
 'San Leandro',
 'South Hayward',
 'South San Francisco',
 'Union City',
 'Walnut Creek',
 'Warm Springs',
 'West Dublin']

In [11]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from stations
group by station
order by station

"""

df1=my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df1

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282
1,16th Street Mission,37.764847,-122.420042,287
2,19th Street,37.807869,-122.268980,67
3,24th Street Mission,37.752000,-122.418700,277
4,Antioch,37.996281,-121.783404,0
5,Ashby,37.853068,-122.269957,299
6,Balboa Park,37.721667,-122.447500,48
7,Bay Fair,37.697000,-122.126500,63
8,Berryessa,37.368361,-121.874655,288
9,Castro Valley,37.690748,-122.075679,0


In [12]:
df2 = df1[df1['station'].isin(find_net_new_customers(10))]

In [13]:
df2


,station,latitude,longitude,transfer_time
4,Antioch,37.996281,-121.783404,0
8,Berryessa,37.368361,-121.874655,288
16,Dublin,37.701663,-121.899232,0
20,Fremont,37.557489,-121.976620,306
28,Milpitas,37.410278,-121.891111,292
35,Pittsburg Center,38.018227,-121.890178,0
45,Union City,37.590746,-122.017282,293
47,Warm Springs,37.503000,-121.940000,278


In [14]:
f = open('gmap_api_key.txt', 'r')
my_api_key = f.read()
f.close()

gmaps.configure(api_key=my_api_key)

In [15]:
colors = []
def make_lines(file_name):
    "clean json"
    
    f = open(file_name, "r")
    j = json.load(f)
    f.close()   
    
    features = []
    names = []
    stroke = {}
    
    
    
    for i in j['features']:
        if i['geometry']['type'] == 'LineString':
            x = i['properties']['name']
            y = re.sub("[\(\[].*?[\)\]]", "", x)
            i['properties']['name'] = y
            if i['properties']['name'] in names:
                pass
            if 'stroke' not in i['properties'].keys():
                pass
            else:
                colors.append(i['properties']['stroke'])
                names.append(i['properties']['name'])
                features.append(i)
                
    stroke['type'] = 'FeatureCollection'            
    stroke['features'] = features          
    
    pprint.pprint(stroke)
    
    with open('stroke.json', 'w', encoding='utf-8') as f:
        json.dump(stroke, f, ensure_ascii=False, indent=4)
       
        

In [16]:
make_lines("transit.json")

{'features': [{'geometry': {'coordinates': [[-122.212207, 37.713272],
                                            [-122.211816, 37.713725],
                                            [-122.211414, 37.714172],
                                            [-122.211005, 37.714614],
                                            [-122.210574, 37.715044],
                                            [-122.210095, 37.71544],
                                            [-122.209557, 37.715784],
                                            [-122.208969, 37.716072],
                                            [-122.20834, 37.716301],
                                            [-122.207695, 37.716498],
                                            [-122.207051, 37.716697],
                                            [-122.206412, 37.716907],
                                            [-122.205774, 37.71712],
                                            [-122.205139, 37.717337],
                       

                                             '1004968, 1004023, 1004083, '
                                             '1004969, 1004024, 1061509, '
                                             '1004970, 1004025, 1061510, '
                                             '1004971, 1004026, 1061511, '
                                             '1004972, 1004027, 1061512, '
                                             '1004973, 1004028, 1061513, '
                                             '1004974, 1061514, 1004029, '
                                             '1004975, 1061515, 1004030, '
                                             '1004976, 1061516, 1004031, '
                                             '1004977, 1061517, 1004032, '
                                             '1004978, 1061518, 1004979, '
                                             '1004033, 1061520, 1004980, '
                                             '1004034, 1061521, 1004981, '
                         

                                            [-122.154039, 37.715784],
                                            [-122.153517, 37.715424],
                                            [-122.152994, 37.715065],
                                            [-122.152471, 37.714706],
                                            [-122.151947, 37.714347],
                                            [-122.151423, 37.713989],
                                            [-122.150899, 37.713631],
                                            [-122.150375, 37.713273],
                                            [-122.14985, 37.712915],
                                            [-122.149326, 37.712557],
                                            [-122.148802, 37.712199],
                                            [-122.148278, 37.71184],
                                            [-122.147754, 37.711482],
                                            [-122.147231, 37.711123],
                      

                                            [-122.183365, 37.742392],
                                            [-122.18384, 37.742791],
                                            [-122.184315, 37.74319],
                                            [-122.18479, 37.74359],
                                            [-122.185264, 37.74399],
                                            [-122.185738, 37.74439],
                                            [-122.186211, 37.74479],
                                            [-122.186684, 37.745191],
                                            [-122.187157, 37.745592],
                                            [-122.18763, 37.745993],
                                            [-122.188103, 37.746393],
                                            [-122.188577, 37.746794],
                                            [-122.18905, 37.747195],
                                            [-122.189524, 37.747595],
                             

               'properties': {'description': 'Trips using this shape (22 in '
                                             'total): 1061252, 1061248, '
                                             '1061224, 1061225, 1061226, '
                                             '1061227, 1061228, 1061229, '
                                             '1061230, 1061231, 1061232, '
                                             '1061233, 1061234, 1061235, '
                                             '1061236, 1061237, 1061238, '
                                             '1061239, 1061240, 1061241, '
                                             '1061242, 1061243',
                              'name': '1061252_shp ',
                              'stroke': '#0099cc',
                              'stroke-opacity': 1,
                              'stroke-width': 3,
                              'styleHash': '-63818bd5',
                              'styleUrl': '#route_Blue-S',
             

...

                                            [-122.062743, 37.917899],
                                            [-122.062978, 37.917383],
                                            [-122.06318, 37.916857],
                                            [-122.063347, 37.916324],
                                            [-122.063476, 37.915785],
                                            [-122.063573, 37.915241],
                                            [-122.063646, 37.914695],
                                            [-122.063708, 37.914148],
                                            [-122.063765, 37.9136],
                                            [-122.063837, 37.913054],
                                            [-122.063927, 37.912509],
                                            [-122.064054, 37.911969],
                                            [-122.06422, 37.911436],
                                            [-122.064424, 37.910911],
                        

                                            [-122.227851, 37.856168],
                                            [-122.228451, 37.855894],
                                            [-122.229052, 37.85562],
                                            [-122.229652, 37.855346],
                                            [-122.230252, 37.855072],
                                            [-122.230852, 37.854797],
                                            [-122.231452, 37.854522],
                                            [-122.232051, 37.854247],
                                            [-122.23265, 37.853971],
                                            [-122.233249, 37.853696],
                                            [-122.233848, 37.853419],
                                            [-122.234446, 37.853141],
                                            [-122.235043, 37.852863],
                                            [-122.235639, 37.852583],
                      

                                            [-122.389224, 37.797029],
                                            [-122.389827, 37.79676],
                                            [-122.39043, 37.79649],
                                            [-122.391033, 37.79622],
                                            [-122.391634, 37.795948],
                                            [-122.392234, 37.795674],
                                            [-122.392831, 37.795397],
                                            [-122.393426, 37.795115],
                                            [-122.394016, 37.794829],
                                            [-122.394596, 37.794529],
                                            [-122.395147, 37.794197],
                                            [-122.395668, 37.793836],
                                            [-122.396172, 37.793459],
                                            [-122.396663, 37.793072],
                        

                                            [-122.410493, 37.782101],
                                            [-122.410978, 37.781709],
                                            [-122.411465, 37.781318],
                                            [-122.411953, 37.780929],
                                            [-122.412442, 37.78054],
                                            [-122.412929, 37.78015],
                                            [-122.413414, 37.779758],
                                            [-122.413898, 37.779366],
                                            [-122.414398, 37.778986],
                                            [-122.414895, 37.778604],
                                            [-122.415389, 37.778219],
                                            [-122.415879, 37.777831],
                                            [-122.416366, 37.777441],
                                            [-122.41685, 37.777049],
                       

                                             '1033074, 1001411, 1053098, '
                                             '1001418, 1033075, 1001443, '
                                             '1053099, 1001442, 1033076, '
                                             '1001426, 1053100, 1001437, '
                                             '1033077, 1001444, 1053101, '
                                             '1001429, 1033078, 1001445, '
                                             '1001462, 1053102, 1033079, '
                                             '1001461, 1001438, 1053103, '
                                             '1033080, 1001414, 1001453, '
                                             '1053104, 1033081, 1001451, '
                                             '1001446, 1053105, 1033082, '
                                             '1001460, 1001456, 1053106, '
                                             '1033083, 1001454, 1001450, '
                         

In [17]:
f = open('stroke.json')
transit = json.load(f)
f.close()

In [20]:
fig = gmaps.figure(map_type = 'HYBRID')

geojson_layer = gmaps.geojson_layer(transit, stroke_color = colors, stroke_weight = 3.5)

fig.add_layer(geojson_layer)

fig.add_layer(gmaps.marker_layer(df2[['latitude','longitude']]))

fig
    

Figure(layout=FigureLayout(height='420px'))

In [ ]:
https://developers.google.com/maps/documentation/javascript/examples/circle-simple